In [1]:
!pwd

/home/sagemaker-user/Brain-Tumor-Classification-MRI/notebooks


In [2]:
%cd ..

/home/sagemaker-user/Brain-Tumor-Classification-MRI


# Callbacks Configuration

In [3]:
from dataclasses import dataclass

In [4]:
@dataclass
class CallbacksConfig:
    callbacks_folder: str
    tensorboard_dir: str
    checkpoint_path: str

# Configuration Manager

In [5]:
from src.utils import read_yaml, create_dirs
from src.constants import *
import os

In [6]:
class ConfigManager:
    def __init__(self, config_path=CONFIG_PATH, params_path=PARAMS_PATH):
        self.config = read_yaml(config_path)
        self.artifacts_folder = self.config.artifacts_folder
        create_dirs([self.artifacts_folder])

    def get_callbacks_config(self):
        config = self.config.callbacks
        tensorboard_dir = os.path.join(
            self.artifacts_folder, config.folder_name, config.tensorboard_folder_name
        )
        checkpoint_path = os.path.join(
            self.artifacts_folder, config.folder_name, config.checkpoint
        )

        create_dirs([
            tensorboard_dir,
            os.path.dirname(checkpoint_path)
        ])

        return CallbacksConfig(
            callbacks_folder = self.config.callbacks,
            tensorboard_dir = tensorboard_dir,
            checkpoint_path = checkpoint_path
        )

# Callbacks Function

In [7]:
import tensorflow as tf
import time

2024-09-28 07:07:20.569819: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-28 07:07:20.597944: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
class Callbacks:
    def __init__(self, config: CallbacksConfig):
        self.config = config

    @property
    def _tensorboard(self):
        timestamp = time.strftime("%Y-%m-%d-T%H-%M-%S")
        logs_dir = os.path.join(self.config.tensorboard_dir, timestamp)
        return tf.keras.callbacks.TensorBoard(logs_dir)

    @property
    def _checkpoint(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_path,
            save_best_only = True
        )

    def get_callbacks(self):
        return [self._tensorboard, self._checkpoint]

# Perform The Step

In [9]:
try:
    config = ConfigManager().get_callbacks_config()
    Callbacks(config).get_callbacks()
except Exception as e:
    raise e